In [31]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from diffprivlib.models import pca, logistic_regression
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np


# Load data

In [18]:
trump_data = pd.read_csv('../data/2016-cces-trump.csv')

# Visualize data


In [19]:
trump_data.head


<bound method NDFrame.head of          uid stateabb  inputstate  race  angryracism  whiteadv  fearraces  \
0          1       NH          33     1          2.0       3.0        1.0   
1          2       LA          22     1          1.0       4.0        1.0   
2          3       MO          29     2          NaN       NaN        NaN   
3          4       AL           1     2          NaN       NaN        NaN   
4          5       CO           8     1          2.0       1.0        1.0   
...      ...      ...         ...   ...          ...       ...        ...   
64595  64596       NY          36     2          NaN       NaN        NaN   
64596  64597       CO           8     1          NaN       NaN        NaN   
64597  64598       LA          22     1          NaN       NaN        NaN   
64598  64599       RI          44     1          NaN       NaN        NaN   
64599  64600       AL           1     1          NaN       NaN        NaN   

       racerare  acograc  aemprac  ...  se_lr

### TODO: Figure out what to do about the NaNs. For now, replacing with N(0,1) noise

# Cleaning data

In [20]:
trump_data.dropna(subset='votetrump')
trump_data.fillna(0, inplace = True)

In [21]:
for col in trump_data.columns:
    values = [v for v in trump_data[col].values if v!=np.nan]

    if isinstance(values[0], str):
        print(str(values[0]))
        map = {v:i for i, v in enumerate(values)}
        trump_data[col].replace(map, inplace=True)

NH
New Hampshire
Northeast
New England
3: Gen X


In [24]:
y = trump_data['votetrump'].to_numpy()
X = trump_data.loc[:, trump_data.columns!='votetrump'].to_numpy()

# Baseline Model

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [38]:
pca = PCA(n_components=5)
tX_train = pca.fit_transform(X_train)
tX_test = pca.transform(X_test)

In [39]:
clf = LogisticRegression(max_iter = 200).fit(tX_train, y_train)

In [41]:
print(f"Performance on training data {clf.score(tX_train, y_train)}")
print(f"Performance on test data {clf.score(tX_test, y_test)}")

Performance on training data 0.711001031991744
Performance on test data 0.7028482972136223
